# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

RESPOSTA
comando:
dbt run --models orders_by_year.sql

query:
{{ config(materialized='table') }}

   SELECT  extract( year from order_approved_at) orders_year , count(1) qtd_orders
   FROM 
      `dbt-shop-melhipolito.dbt_shop.orders` 
   WHERE 
      order_approved_at is not null
   GROUP BY extract( year from order_approved_at)

imagem:
![exercicio_1](imagens_atividade_03/exercicio_1.png)   

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

RESPOSTA:

comando:
dbt run --models view_orders.sql

query:
{{ config(materialized='view') }}

   select count(1) orders_total, c.customer_state
    from dbt_shop.customers c
    inner join dbt_shop.orders o on c.customer_id = o.customer_id
    group by c.customer_state
    order by c.customer_state

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

RESPOSTA:

comando:
dbt run --models view_orders.sql --vars "{'number_of_orders': 5000}"

query:
{{ config(materialized='view') }}

   select * from (
      select count(1) orders_total, c.customer_state
      from dbt_shop.customers c
      inner join dbt_shop.orders o on c.customer_id = o.customer_id
      group by c.customer_state
      order by c.customer_state) t
   where t.orders_total  > cast('{{ var("number_of_orders") }}' as int64)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

RESPOSTA:

comando:
dbt run --models customers_by_region.sql

query:
{{ config(materialized='view') }}

   select count(1) customers_total,
       case when c.customer_state in ('SP','MG','ES','RJ') then
        'Sudeste'
       when c.customer_state in ('PR','RS','SC') then
        'Sul'
       when c.customer_state in ('AM','RR','PA','AM','TO','RO','AC') then       
        'Norte'
       when c.customer_state in ('CE','PI','MA','BA','RN','PB','PE','AL','SE') then
        'Nordeste'
       else
        'Centro Oeste' end
       as region
    from dbt_shop.customers c
    group by case when c.customer_state in ('SP','MG','ES','RJ') then
        'Sudeste'
       when c.customer_state in ('PR','RS','SC') then
        'Sul'
       when c.customer_state in ('AM','RR','PA','AM','TO','RO','AC') then       
        'Norte'
       when c.customer_state in ('CE','PI','MA','BA','RN','PB','PE','AL','SE') then
        'Nordeste'
       else
        'Centro Oeste' end


5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

RESPOSTA:

comando:
 dbt run --models items_costs.sql   

query:
{{ config(materialized='table') }}

   select (i.freight_value + i.price) item_price,
       i.product_id
    from dbt_shop.items i
    order by i.product_id

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

RESPOSTA:

comando:
dbt run --models orders_shipped_costs.sql

query:
{{ config(materialized='view') }}

   select trunc(sum(ic.item_price ),2) order_costs,
       o.order_id
    from dbt_shop.orders o,
        dbt_shop.items i,
        dataset_melhipolito.items_costs ic
    where o.order_status = 'shipped'
    and   o.order_id = i.order_id
    and   i.product_id = ic.product_id
    group by o.order_id